In [ ]:
# Hi there, welcome to NOTEBOOK 1: Preparing data, in this notebook we will prepare our datasets for the next steps.

import pandas as pd


# Load a .tsv file into a DataFrame
file_path = # Replace with your file path to 'Notes' dataset  
df_notes_dataset = pd.read_csv(file_path, sep='\\t')


# Remove columns by their indexes
columns_to_remove = [1, 3, 5, 6, 7, 22]  
df_notes_dataset = df_notes_dataset.drop(df_notes_dataset.columns[columns_to_remove], axis=1)

# Display the updated DataFrame
df_notes_dataset.head()

In [ ]:
# Load a .tsv file into a DataFrame
file_path = # Replace with your file path to 'Note Status History' dataset 
df_status_history_dataset = pd.read_csv(file_path, sep='\\t')


# Remove columns by their indexes
columns_to_remove = [1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]  # Replace with the indexes of the columns you want to remove
df_status_history_dataset = df_status_history_dataset.drop(df_status_history_dataset.columns[columns_to_remove], axis=1)

# Display the updated DataFrame
df_status_history_dataset.info()

In [ ]:
# Load a .tsv file into a DataFrame
file_path = # Replace with your file path to 'Ratings' dataset
df_ratings_dataset = pd.read_csv(file_path, sep='\\t')


# Remove columns by their indexes
columns_to_remove = [1, 3, 6, 7, 9, 10, 11, 12, 14, 15, 17, 18, 19, 22, 23, 25, 26, 27, 28, 29, 30, 31] 
df_ratings_dataset = df_ratings_dataset.drop(df_ratings_dataset.columns[columns_to_remove], axis=1)

# Display the updated DataFrame
df_ratings_dataset.info()

In [ ]:
# Find the intersection of noteId values across all three datasets
common_note_ids = set(df_notes_dataset['noteId']) & set(df_status_history_dataset['noteId']) & set(df_ratings_dataset['noteId'])

# Filter each dataset to keep only rows with noteId values in the intersection
df_notes_dataset = df_notes_dataset[df_notes_dataset['noteId'].isin(common_note_ids)]
df_status_history_dataset = df_status_history_dataset[df_status_history_dataset['noteId'].isin(common_note_ids)]
df_ratings_dataset = df_ratings_dataset[df_ratings_dataset['noteId'].isin(common_note_ids)]

# Display the updated DataFrames
print(f"Filtered df_notes_dataset: {df_notes_dataset.shape}")
print(f"Filtered df_status_history_dataset: {df_status_history_dataset.shape}")
print(f"Filtered df_ratings_dataset: {df_ratings_dataset.shape}")

In [ ]:
# Export the cleaned DataFrames to CSV files

df_notes_dataset.to_csv(# File path Notes dataset clean_1, index=False)
df_status_history_dataset.to_csv(# File path Note Status History dataset clean_1, index=False)
df_ratings_dataset.to_csv(# File path  Ratings dataset clean_1, index=False)

In [ ]:
# We load in the previously cleaned datasets for further processing and analysis as new Dataframes

df_notes = pd.read_csv(#input file path to Notes dataset clean_1)
df_status_history = pd.read_csv(#input file path to Note Status History dataset clean_1)
df_ratings = pd.read_csv(#input file path to Ratings dataset clean_1)

In [ ]:
# In this step, we filter the datasets to include only entries created in 2024 and later, which is usful for my particular analysis.

# Define the timestamp for the start of 2024 (in milliseconds)

start_of_2024 = 1704067200000  # January 1, 2024, in milliseconds

# Filter df_notes_dataset
df_notes['createdAtMillis'] = pd.to_numeric(df_notes['createdAtMillis'], errors='coerce')
df_notes = df_notes[df_notes['createdAtMillis'] >= start_of_2024]

# Filter df_status_history_dataset
df_status_history['createdAtMillis'] = pd.to_numeric(df_status_history ['createdAtMillis'], errors='coerce')
df_status_history = df_status_history [df_status_history ['createdAtMillis'] >= start_of_2024]

# Filter df_ratings_dataset
df_ratings['createdAtMillis'] = pd.to_numeric(df_ratings['createdAtMillis'], errors='coerce')
df_ratings = df_ratings[df_ratings['createdAtMillis'] >= start_of_2024]

# Display the updated DataFrames
print(f"Filtered df_notes_dataset: {df_notes.shape}")
print(f"Filtered df_status_history_dataset: {df_status_history.shape}")
print(f"Filtered df_ratings_dataset: {df_ratings.shape}")

In [ ]:
# Export the version 2 of the cleaned DataFrames to CSV files, these include a specific timeframe filter.

df_notes.to_csv(# File path Notes dataset clean_2, index=False)
df_status_history.to_csv(# File path Note Status History dataset clean_2, index=False)
df_ratings.to_csv(# File path  Ratings dataset clean_2, index=False)

In [ ]:
# Now we are going to extract the URLs from the Notes datasets which can be identified with the URLs of European fact-checking organisations. 

# We use a regular expression to find URLs in the 'summary' column of the Notes dataset and create a new column 'sourceURL' with the extracted URLs.
import re

df_notes = pd.read_csv(# input file path to Notes dataset clean_2)
    
# Define a function to extract URLs from a string
def extract_urls(text):
    if pd.isna(text):  # Handle NaN values
        return None
    # Regular expression to match URLs
    url_pattern = r'(https?://[^\s]+)'
    urls = re.findall(url_pattern, text)
    return urls[0] if urls else None  # Return the first URL found, or None if no URL is found

# Apply the function to the 'summary' column and create a new column 'sourceURL'
df_notes['sourceURL'] = df_notes['summary'].apply(extract_urls)

# Drop rows where 'sourceURL' is NaN (i.e., rows without a URL)
df_notes = df_notes.dropna(subset=['sourceURL'])

# Display the updated DataFrame
df_notes.info()

In [ ]:
# Now we use tldextract to extract the domain and suffix from the 'sourceURL' column and add it to a new column 'domainExtract'.

import tldextract

# Extract the domain and suffix from the sourceURL column and add it to a new column 'domainExtract'
df_notes['domainExtract'] = df_notes['sourceURL'].apply(lambda url: f"{tldextract.extract(url).domain}.{tldextract.extract(url).suffix}")

# Display the updated DataFrame
df_notes[['sourceURL', 'domainExtract']].head(10)

In [ ]:
# Now we match the 'domainExtract' column with a predefined list of domains to identify the source of the Notes.

# Define the list of domains to match
EMDO_members = [
    "afp.com",
    "correctiv.org",
    "delfi.lt",
    "demagog.org.pl",
    "dpa.com",
    "verifica.efe.com",
    "thejournal.ie",
    "faktabaari.fi",
    "faktisk.no",
    "ellinikahoaxes.gr",
    "kallkritikbyran.se",
    "knack.be",
    "lessurligneurs.eu",
    "maldita.es",
    "mimikama.org",
    "newsguardtech.com",
    "newtral.es",
    "ostro.si",
    "pagellapolitica.it",
    "poligrafo.sapo.pt",
    "science.feedback.org",
    "tjekdet.dk",
    "rtve.es",
    "verificat.cat"
]

EFCSN_members = [
    "hibrid.info",
    "afp.com",
    "apa.at",
    "civilnet.am",
    "correctiv.org",
    "delfi.lt",
    "demagog.org.pl",
    "demagog.cz",
    "demagog.sk",
    "dogrula.org",
    "dpa.com",
    "verifica.efe.com",
    "ellinikahoaxes.gr",
    "factcheckcyprus.org",
    "facta.news",
    "factcheck.ge",
    "factcheck.bg",
    "factcheck.vlaanderen",
    "factcheckni.org",
    "factreview.gr",
    "factual.ro",
    "fakenews.rs",
    "faktisk.no",
    "faktograf.hr",
    "faktoje.al",
    "fullfact.org",
    "maldita.es",
    "geofacts.ee",
    "factchecker.gr",
    "info-veritas.com",
    "kallxo.com",
    "istinomer.rs",
    "istinomjer.ba",
    "kallkritikbyran.se",
    "knack.be",
    "lakmusz.hu",
    "leadstories.com",
    "lessurligneurs.eu",
    "lupa.lupiga.com",
    "medizin-transparent.at",
    "mythdetector.com",
    "newtral.es",
    "ostro.si",
    "pa.media",
    "pagellapolitica.it",
    "15min.lt",
    "poligrafo.sapo.pt",
    "pravda.org.pl",
    "provereno.media",
    "raskrikavanje.rs",
    "raskrinkavanje.ba",
    "raskrinkavanje.me",
    "rebaltica.lv",
    "science.feedback.org",
    "teyit.org",
    "tjekdet.dk",
    "verificat.cat",
    "viral.sapo.pt",
    "vistinomer.mk",
    "vrt.be",
    "investigatebel.org",
    
]

# Create regex patterns for EFCSN and EDMO members
efcsn_pattern = '|'.join(EFCSN_members)
edmo_pattern = '|'.join(EMDO_members)

# Add columns to indicate whether the domain is part of EFCSN, EDMO, or Other Source
df_notes['noteFromEFCSN'] = df_notes['domainExtract'].str.contains(efcsn_pattern, na=False)
df_notes['noteFromEDMO'] = df_notes['domainExtract'].str.contains(edmo_pattern, na=False)
df_notes['noteFromOtherSource'] = ~(
    df_notes['noteFromEFCSN'] | df_notes['noteFromEDMO']
)

# Convert boolean columns to integers (optional, if needed)
df_notes['noteFromEFCSN'] = df_notes['noteFromEFCSN'].astype(int)
df_notes['noteFromEDMO'] = df_notes['noteFromEDMO'].astype(int)
df_notes['noteFromOtherSource'] = df_notes['noteFromOtherSource'].astype(int)

# Count the number of true values in each boolean column
true_counts = {
    "noteFromEFCSN": df_notes['noteFromEFCSN'].sum(),
    "noteFromEDMO": df_notes['noteFromEDMO'].sum(),
    "noteFromOtherSource": df_notes['noteFromOtherSource'].sum()
}

# Display the counts
print(true_counts)

# Save the updated DataFrame to a new CSV file
df_notes.to_csv(# File path Notes dataset clean_3, index=False)

In [ ]:
# We print the total number of unique notes that are from either EFCSN or EDMO or both, so we know how many unique notes are from these sources.

# Get notes that are from either EFCSN or EDMO or both
either_efcsn_or_edmo = df_notes[(df_notes['noteFromEFCSN'] == 1) | (df_notes['noteFromEDMO'] == 1)]

# Count unique noteIds
unique_note_count = either_efcsn_or_edmo['noteId'].nunique()

print(f"Total number of unique notes from either EFCSN or EDMO members: {unique_note_count}")

In [ ]:
# THIS IS THE END OF NOTEBOOK CLEANING AND PROCESSING

# THE MANUAL ANNOTATION OF THE DATASETS WILL BE DONE IN NOTEBOOK 2